In [2]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1

     |████████████████████████████████| 551.3MB 29kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 245kB 9.3MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.8.3-cp36-none-any.whl size=293540 sha256=1e5eb0858eed1d4ca31bb0178e0d6ecb4af3174df9ebd41278624824c3910dab
  Stored in directory: /root/.cache/pip/wheels/50/6e/32/521aa84da7f9ee725d3c9be0b5e0d771df659bf25da5929f6c
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 9.1MB/s 
     |████████████████████████████████| 317kB 8.6MB/s 
     |████████████████████████████████| 870kB 6.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=e7ac566814c7648e5a7a7aeec75add191406aef841048f691ff19d5d1b05b9d7
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |█

In [1]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ggs1d9y2
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ggs1d9y2
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12854 sha256=0ebadd8fae7fc190287c85109b773c8262b5b929fd805370636d044e4bd674f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-5keqso_4/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [0]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [3]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [0]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

In [0]:
##GPU 사용 시
device = torch.device("cuda:0")

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [7]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2020-02-10 12:56:37--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2020-02-10 12:56:37--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc53b2525bff76b264ec3133f11c.dl.dropboxusercontent.com/cd/0/get/Ax2rnLPnlD7bXNjxQdo5CJTATdLYeXFHomI3X1kqzVY8FKCdmDEX8Mlj15o8C4icg1re4Zrgq9iQc-xugRJH9q4E61RhXZf1VSSeq28tkVs1FWcOWLPn3tUtHs9wZXGaY9I/file?dl=1# [following]
--2020-02-10 12:56:38--  https://uc53b2525bff76b264ec3133f11c.dl.dropboxusercontent.com/cd/0/get/Ax2rnLPnlD7bXNjxQdo5CJTATdLYeXFHomI3X1kqzVY8FKCdmDEX8Mlj15o8C4icg1re4Zrgq9iQc-xugRJH9q4E61RhXZ

In [0]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [9]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [0]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [0]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [0]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [0]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [0]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [0]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [0]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [0]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [0]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [0]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

In [0]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [22]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1 loss 0.7354031801223755 train acc 0.421875
epoch 1 batch id 201 loss 0.4972061514854431 train acc 0.5680192786069652
epoch 1 batch id 401 loss 0.48264944553375244 train acc 0.6792394014962594
epoch 1 batch id 601 loss 0.4473280608654022 train acc 0.7324511231281198
epoch 1 batch id 801 loss 0.3884296715259552 train acc 0.7621722846441947
epoch 1 batch id 1001 loss 0.32982003688812256 train acc 0.780531968031968
epoch 1 batch id 1201 loss 0.3920602798461914 train acc 0.7934273522064946
epoch 1 batch id 1401 loss 0.38829025626182556 train acc 0.8029644004282656
epoch 1 batch id 1601 loss 0.36050739884376526 train acc 0.8111239069331667
epoch 1 batch id 1801 loss 0.2709638774394989 train acc 0.818052470849528
epoch 1 batch id 2001 loss 0.25238534808158875 train acc 0.8239005497251374
epoch 1 batch id 2201 loss 0.26074808835983276 train acc 0.8290762721490231

epoch 1 train acc 0.8324334293230944



epoch 1 test acc 0.8779571611253197


epoch 2 batch id 1 loss 0.5016367435455322 train acc 0.796875
epoch 2 batch id 201 loss 0.22650308907032013 train acc 0.8842506218905473
epoch 2 batch id 401 loss 0.27612945437431335 train acc 0.8854426433915212
epoch 2 batch id 601 loss 0.3465675711631775 train acc 0.8902350249584027
epoch 2 batch id 801 loss 0.3841121196746826 train acc 0.8924196317103621
epoch 2 batch id 1001 loss 0.27524134516716003 train acc 0.8944649100899101
epoch 2 batch id 1201 loss 0.2626759111881256 train acc 0.897051935886761
epoch 2 batch id 1401 loss 0.21202902495861053 train acc 0.8996475731620271
epoch 2 batch id 1601 loss 0.2541404366493225 train acc 0.9015263897564022
epoch 2 batch id 1801 loss 0.16796402633190155 train acc 0.9036733064963909
epoch 2 batch id 2001 loss 0.1809045970439911 train acc 0.9057892928535732
epoch 2 batch id 2201 loss 0.2921770513057709 train acc 0.9071515788278055

epoch 2 train acc 0.9081986810295791



epoch 2 test acc 0.8924432544757033


epoch 3 batch id 1 loss 0.33769428730010986 train acc 0.859375
epoch 3 batch id 201 loss 0.12563379108905792 train acc 0.923818407960199
epoch 3 batch id 401 loss 0.16506363451480865 train acc 0.9259663341645885
epoch 3 batch id 601 loss 0.22633811831474304 train acc 0.9295445091514143
epoch 3 batch id 801 loss 0.2125745415687561 train acc 0.9314723782771536
epoch 3 batch id 1001 loss 0.20969116687774658 train acc 0.9334571678321678
epoch 3 batch id 1201 loss 0.12654508650302887 train acc 0.9358997710241466
epoch 3 batch id 1401 loss 0.17518197000026703 train acc 0.9378122769450392
epoch 3 batch id 1601 loss 0.1782539039850235 train acc 0.9389834478450968
epoch 3 batch id 1801 loss 0.1297226846218109 train acc 0.9401547751249306
epoch 3 batch id 2001 loss 0.13032497465610504 train acc 0.9414121064467766
epoch 3 batch id 2201 loss 0.1762569695711136 train acc 0.9422279645615629

epoch 3 train acc 0.9429083120022754



epoch 3 test acc 0.8937619884910486


epoch 4 batch id 1 loss 0.43784481287002563 train acc 0.828125
epoch 4 batch id 201 loss 0.04113700985908508 train acc 0.9539800995024875
epoch 4 batch id 401 loss 0.11320450901985168 train acc 0.9549953241895262
epoch 4 batch id 601 loss 0.2407946139574051 train acc 0.9568427620632279
epoch 4 batch id 801 loss 0.15168030560016632 train acc 0.9588600187265918
epoch 4 batch id 1001 loss 0.2006414532661438 train acc 0.9600867882117882
epoch 4 batch id 1201 loss 0.03155696019530296 train acc 0.9611521648626145
epoch 4 batch id 1401 loss 0.05618217960000038 train acc 0.9623148643825838
epoch 4 batch id 1601 loss 0.1327083259820938 train acc 0.9627771705184259
epoch 4 batch id 1801 loss 0.030694937333464622 train acc 0.9635879372570794
epoch 4 batch id 2001 loss 0.11989239603281021 train acc 0.9643849950024987
epoch 4 batch id 2201 loss 0.12422765791416168 train acc 0.9647887323943662

epoch 4 train acc 0.9651948236632536



epoch 4 test acc 0.8969988810741688


epoch 5 batch id 1 loss 0.44691863656044006 train acc 0.875
epoch 5 batch id 201 loss 0.023489845916628838 train acc 0.9748134328358209
epoch 5 batch id 401 loss 0.049125343561172485 train acc 0.9745557980049875
epoch 5 batch id 601 loss 0.2163194715976715 train acc 0.9752235856905158
epoch 5 batch id 801 loss 0.06841138750314713 train acc 0.9758309925093633
epoch 5 batch id 1001 loss 0.013305937871336937 train acc 0.9765234765234765
epoch 5 batch id 1201 loss 0.01062062382698059 train acc 0.9768942547876769
epoch 5 batch id 1401 loss 0.04662745073437691 train acc 0.9770476445396146
epoch 5 batch id 1601 loss 0.11756110191345215 train acc 0.9772407870081199
epoch 5 batch id 1801 loss 0.01136554405093193 train acc 0.9776773320377568
epoch 5 batch id 2001 loss 0.09791291505098343 train acc 0.9780656546726637
epoch 5 batch id 2201 loss 0.11551730334758759 train acc 0.9780923443889141

epoch 5 train acc 0.9782045470705347



epoch 5 test acc 0.8968989769820972
